In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb


We will be looking at school specific data from across the United States. Our goal is to see if any schools resemble eachother. Often schools are described as "good", but if you are not familiar with the area you have little idea what that definition of "good" means. We want to explore the school graduation rates, diversity, expulsion rates, and teacher ratios to get an idea of what a "good" school is.

If we are also able to cluster like schools together, you may be able to see how your current school (something you are familiar with) compares to other schools you are looking to move into.

## Data sources:

2015-2016 graduation rates: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr

2015-2016 school location, characteristics, staff, and lunch programs: https://nces.ed.gov/ccd/pubschuniv.asp

2015-2016 school achievement: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr

2015-2016 suspenion rates: https://ocrdata.ed.gov/DataAnalysisTools/DataSetBuilder?Report=4
for this analysis, we only focused on the greenville district schools, but you could select other areas of interest.
    

In [2]:
#load data
path='/home/natalie/Documents/SchoolAPI/Data_Sources/'
grad_2015=pd.read_csv(path+'graduation_rates/2015_school_graduation.csv', parse_dates=True) #high school graduation rates


In [3]:
#school specific data
#school_lunch=pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_lunches_ccd_sch_033_1516.csv',  parse_dates=True)
#school_char=pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_characteristics_ccd_sch_129_1516_w_2a_011717.csv', parse_dates=True)
school_dir =pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_directory_ccd_sch_029_1516_w_2a_011717.csv',  parse_dates=True)
school_member=pd.read_csv(path+ 'CCD_comoncoredata_nonfiscal/schools_membership_ccd_sch_052_1516_w_2a_011717.csv' ,  parse_dates=True)
school_staff = pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_staff_ccd_sch_059_1516_w_2a_011717.csv')

/home/natalie/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,14,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/natalie/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#School achievement results for math and reading

math=pd.read_csv(path+'Subject_assesments/school_math-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)
reading= pd.read_csv(path+'Subject_assesments/rla-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)

# percent is the % above the state's "proficient" level
#colums starting with F or M denote breakdowns by gender


In [5]:
#greenville suspensions
suspensions=pd.read_csv(path+'civil_rights/suspensions_greenville_2015.csv')

In [6]:
suspensions=suspensions.replace('%', '', regex=True)
#suspensions.info()

In [7]:
#n = individual counts of students
#otherwise in percentage
#times white students is a multiplier of how much more likely they are to be suspended than a white student

#make sure objects are now numeric
suspensions['TOT_OSS_rate']=suspensions['TOT_OSS_rate'].astype(str).astype(float)
suspensions['BLA_OSS_rate']=suspensions['BLA_OSS_rate'].astype(str).astype(float)
suspensions['WHI_OSS_rate']=suspensions['WHI_OSS_rate'].astype(str).astype(float)

In [8]:
suspensions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 35 columns):
District ID               92 non-null int64
District                  92 non-null object
State                     92 non-null object
School_ID                 92 non-null int64
School Name               92 non-null object
AME_n_susp                92 non-null int64
ASI_n_susp                92 non-null int64
HI PAC_n_susp             92 non-null int64
BLA_n_susp                92 non-null int64
HIS_n_susp                92 non-null int64
WHI_n_susp                92 non-null int64
TWO_n_susp                92 non-null int64
TOT_n_susp                92 non-null int64
AME_OSS_rate              92 non-null object
ASI_OSS_rate              92 non-null object
HI_PAC_OSS_rate           92 non-null object
BLA_OSS_rate              92 non-null float64
HIS_OSS_rate              92 non-null object
WHI_OSS_rate              92 non-null float64
TWO_OSS_rate              92 non-null object
TOT_O

## Lets pair down the data to only include the variables we are interested in, and then do the cleaning.

### For the math and reading dataste,these variables are:

STNAM = state name

LEANM = education agency name

NCESSCH = unique school ID

SCHNAM = school name

ALL_MTH00NUMVALID_1516  = number of students aggregated across all grades who took test
ALL_MTH00PCTPROF_1516= percentage of all students who are proficient in math across all grades

MWH_MTH00PCTPROF_1516 = percentage of all white students in school proficient in math
MBL_MTH00PCTPROF_1516 = percentage of all black students in school proficient in math

F_MTH00NUMVALID_1516 = females who took math exam across all grades
F_MTH00PCTPROF_1516 = females proficient in math across all grades

M_MTH00NUMVALID_1516 = males who took math exam across all grades
M_MTH00PCTPROF_1516 = males proficient in math across all grades

MBL_MTHHSNUMVALID_1516 = number of black students in high school who took math examns
MBL_MTHHSPCTPROF_1516 = percent of black students in high school who are proficient in math

MWH_MTHHSNUMVALID_1516 = Number of white students in high school who took math exams
MWH_MTHHSPCTPROF_1516 = percent of white students in high school who are proficient in math

same thing for reading, but MTH is replaced with RLA

In [9]:
#keep only variables listed above

math_keep=['STNAM', 'LEANM', 'NCESSCH', 'SCHNAM', 'ALL_MTH00NUMVALID_1516', 'ALL_MTH00PCTPROF_1516', 'MWH_MTH00PCTPROF_1516', 'MBL_MTH00PCTPROF_1516','F_MTH00NUMVALID_1516', 'F_MTH00PCTPROF_1516', 'M_MTH00NUMVALID_1516', 'M_MTH00PCTPROF_1516', 'MBL_MTHHSNUMVALID_1516', 'MBL_MTHHSPCTPROF_1516' , 'MWH_MTHHSNUMVALID_1516', 'MWH_MTHHSPCTPROF_1516']
math=math[math_keep]


reading_keep=['STNAM', 'LEANM', 'NCESSCH', 'SCHNAM', 'ALL_RLA00NUMVALID_1516', 'ALL_RLA00PCTPROF_1516', 'F_RLA00NUMVALID_1516', 'F_RLA00PCTPROF_1516', 'M_RLA00NUMVALID_1516', 'M_RLA00PCTPROF_1516', 'MBL_RLAHSNUMVALID_1516', 'MBL_RLAHSPCTPROF_1516' , 'MWH_RLAHSNUMVALID_1516', 'MWH_RLAHSPCTPROF_1516']
reading=reading[reading_keep]

math.head()

,STNAM,LEANM,NCESSCH,SCHNAM,ALL_MTH00NUMVALID_1516,ALL_MTH00PCTPROF_1516,MWH_MTH00PCTPROF_1516,MBL_MTH00PCTPROF_1516,F_MTH00NUMVALID_1516,F_MTH00PCTPROF_1516,M_MTH00NUMVALID_1516,M_MTH00PCTPROF_1516,MBL_MTHHSNUMVALID_1516,MBL_MTHHSPCTPROF_1516,MWH_MTHHSNUMVALID_1516,MWH_MTHHSPCTPROF_1516
0,ALABAMA,Albertville City,10000500870,Ala Avenue Middle Sch,707,36,45,LE20,347,39,360,34,NaN,NaN,NaN,NaN
1,ALABAMA,Albertville City,10000500871,Albertville High Sch,307,15,20-24,LT50,161,15-19,146,10-14,9,LT50,177,20-24
2,ALABAMA,Albertville City,10000500879,Evans Elem Sch,772,43,52,21-39,389,46,383,40,NaN,NaN,NaN,NaN
3,ALABAMA,Albertville City,10000500889,Albertville Elem Sch,852,55,67,LE20,421,54,431,57,NaN,NaN,NaN,NaN
4,ALABAMA,Marshall County,10000600193,Kate Duncan Smith DAR Middle,443,36,37,PS,206,30-34,237,35-39,NaN,NaN,NaN,NaN


### School directory dataset, we are interested in these variables:
    
FIPST = state name
NCESSCH = school ID
LCITY = location city
LZIP = location zip code
WEBSITE = website
G1OFFERED = grade 1 offered
G8OFFERED = grade 8 offered
G12FFERED = grade 12 offered
NOGRADES = no grades offered, if true this school is not operational


In [10]:
dir_keep=['FIPST', 'NCESSCH', 'LCITY', 'LZIP', 'G1OFFERED', 'G8OFFERED', 'G12OFFERED' ,'NOGRADES']
school_dir=school_dir[dir_keep]

### School membership

This is breakdown in grade by race and gender

NCESSCH = school ID
SCH_NAME = school name
GO1 = number of students in grade 1
GO8 = number of students in grade 8
G12 = number of students in grade 12
TOTAL = total students in all grades in school
AM = number of all students in school american indian
AMALM = '' male
AMALF = "" female
AS = number of all students asian
ASALM = "" male
ASALF = "" female
HI = number of all students Hispanic
HIALM
HIALF
BL = number of all students black
BLALM
BLALF
WH = number of all stduents white
WHALM
WHALF
HP = number of all students hawaiian
HPALM
HPALF
TR = number of all students 2 or more races
TRALM
TRALF

AS01M = american indian Male
AS01F = american indian female
HI01M = hispanic Male
HI01F = hispanic female grade1
BL01M = black male grade 1
BL01F = black Female grade1
WH01M = white male grade 1
WH01F = white female grade 1
HP01M = hawaiian/pacific islander male
HP01F = hawaiian/pacific islander Female
TR01M = two or more races male grade 1
TR01F = two or more races female grade 1



In [11]:
school_mem_keep=['NCESSCH', 'SCH_NAME', 'G01', 'G08', 'G12', 'TOTAL', 'AM', 'AMALM', 'AMALF', 'AS', 'ASALM', 'ASALF', 'HI', 'HIALM', 'HIALF', 'BL', 'BLALM', 'BLALF', 'WH', 'WHALM', 'WHALF', 'HP', 'HPALM', 'HPALF', 'TR', 'TRALM', 'TRALF']
school_member=school_member[school_mem_keep]

### School staff dataset, we are interested in these variables:

NCESSCH = school ID
FTE = full time equivalent teachers (value -1 is missing)

In [12]:
school_staff=school_staff[['NCESSCH','FTE']]

### Graduation rate dataset

NCESSCH = school ID

MAM = American Indian/Alaska Native students

MAS = Asian/Pacific Islander

MHI = Hispanic/Latino

MBL = Black

MWH = White

MTR = Two or more races

CWD = disabilities

ECD = Economically disadvanted

LEP = Limited English Proficiency

COHORT = number of students

RATE = percentage in that group out of the entire student population of that school

In [13]:
grad_keep=['NCESSCH', 'ALL_COHORT_1516', 'ALL_RATE_1516', 'MAM_RATE_1516', 'MAS_RATE_1516', 'MBL_RATE_1516', 'MHI_RATE_1516', 'MTR_RATE_1516', 'MWH_RATE_1516', 'CWD_RATE_1516', 'ECD_RATE_1516', 'LEP_RATE_1516']
grad_2015=grad_2015[grad_keep]

### Suspensions dataset

School_ID = school ID
TOT_n_susp = total number of suspensions in that school
TOT_OSS_rate = total number of suspensions in school/ school enrollment

" " OSS_Rate = number of students in this race suspended/ total student of that race enrollment in school

" " OSS_times_white = the rate of suspensions for this race is xx times that of white students

In [14]:
susp_keep=['School_ID', 'TOT_n_susp', 'TOT_OSS_rate', 'BLA_OSS_rate', 'WHI_OSS_rate', 'AME_OSS_times_white', 'ASI_OSS_times_white', 'HI_PAC_OSS_times_white', 'BLA_OSS_times_white', 'HIS_OSS_times_white', 'TWO_OSS_times_white', 'TOT_n_enrollment']

suspensions=suspensions[susp_keep]

In [15]:
suspensions.head()

,School_ID,TOT_n_susp,TOT_OSS_rate,BLA_OSS_rate,WHI_OSS_rate,AME_OSS_times_white,ASI_OSS_times_white,HI_PAC_OSS_times_white,BLA_OSS_times_white,HIS_OSS_times_white,TWO_OSS_times_white,TOT_n_enrollment
0,450231000049,25,3.78,3.28,4.14,0.0,0.00,0.0,0.79,0.00,0.00,671
1,450231000114,16,1.72,6.74,1.49,0.0,0.00,0.0,4.52,0.00,0.00,960
2,450231000449,29,2.94,5.71,1.76,0.0,28.41,0.0,3.24,1.84,0.00,1017
3,450231000455,94,9.55,17.62,6.50,0.0,0.83,0.0,2.71,0.77,3.62,1025
4,450231000459,72,6.43,17.09,4.82,0.0,0.00,0.0,3.55,1.36,3.53,1164


# Data Cleaning

#### 1. the rates of graduation or math/reading competence can be given in values '50-60', or 'GE50' = greater than 50%, or 'PS' which means the data was intentially hidden so you could not focus in on a very small group (e.g., 2 students are hispanic, so any data on these 2 students would wrongly expose their identity). To be conservative, we will select the lower range 50-60, we would select '50' to represent the graduation rate. For values with GE greater than or LE less than, we simply select the value stated, so GE80 becomes '80'.

In [16]:
import re
grad_variables=list(grad_2015)


In [17]:
grad_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 12 columns):
NCESSCH            23090 non-null int64
ALL_COHORT_1516    23090 non-null int64
ALL_RATE_1516      23090 non-null object
MAM_RATE_1516      19434 non-null object
MAS_RATE_1516      20357 non-null object
MBL_RATE_1516      21736 non-null object
MHI_RATE_1516      22034 non-null object
MTR_RATE_1516      20705 non-null object
MWH_RATE_1516      22716 non-null object
CWD_RATE_1516      22656 non-null object
ECD_RATE_1516      22936 non-null object
LEP_RATE_1516      20740 non-null object
dtypes: int64(2), object(10)
memory usage: 2.1+ MB


In [18]:
replace_list=['GE', 'LE', 'LT'] #remove these Greater than (GE) letters and retain numeric representation only

for word in replace_list:
    reading=reading.replace(word, '', regex=True)
    math=math.replace(word, '', regex=True)
    grad_2015=grad_2015.replace(word, '', regex=True) #regex=TRUE checks for partial match, otherwise it would need to match it 100%


In [19]:
na_list=['.', 'PS']

for word in na_list:
    reading=reading.replace(word, np.nan)
    math=math.replace(word, np.nan)
    grad_2015=grad_2015.replace(word, np.nan) #here we want an exact match, so we do not include reges

In [20]:
#removehe the ranges, instead select the lowest value in the range, if rate is 80-85, we only want to keep 80
#grad_2015['ALL_RATE_1516'] = grad_2015['ALL_RATE_1516'].str.split('-').str[0]

r=re.compile(".*RATE")#finds all variables that end in 'Rate'
rate_variables = list(filter(r.match, grad_variables)) # Read Note

for variable in rate_variables:
    grad_2015[variable]= grad_2015[variable].str.split('-').str[0]

r=re.compile(".*PROF")
prof_variables=list(filter(r.match, list(math)))

for variable in prof_variables:
    math[variable]= math[variable].str.split('-').str[0]
    
#same thing for reading
r=re.compile(".*PROF")
prof_variables=list(filter(r.match, list(reading)))

for variable in prof_variables:
    reading[variable]= reading[variable].str.split('-').str[0]


In [21]:
reading.head(7)

,STNAM,LEANM,NCESSCH,SCHNAM,ALL_RLA00NUMVALID_1516,ALL_RLA00PCTPROF_1516,F_RLA00NUMVALID_1516,F_RLA00PCTPROF_1516,M_RLA00NUMVALID_1516,M_RLA00PCTPROF_1516,MBL_RLAHSNUMVALID_1516,MBL_RLAHSPCTPROF_1516,MWH_RLAHSNUMVALID_1516,MWH_RLAHSPCTPROF_1516
0,ALABAMA,Albertville City,10000500870,Ala Avenue Middle Sch,687,33,339,39,348,28,NaN,NaN,NaN,NaN
1,ALABAMA,Albertville City,10000500871,Albertville High Sch,307,31,162,30,145,25,9,50,178,35
2,ALABAMA,Albertville City,10000500879,Evans Elem Sch,759,30,381,36,378,23,NaN,NaN,NaN,NaN
3,ALABAMA,Albertville City,10000500889,Albertville Elem Sch,839,36,414,37,425,34,NaN,NaN,NaN,NaN
4,ALABAMA,Marshall County,10000600193,Kate Duncan Smith DAR Middle,442,42,205,45,237,35,NaN,NaN,NaN,NaN
5,ALABAMA,Marshall County,10000600872,Asbury Sch,297,27,139,35,158,15,NaN,NaN,46,20
6,ALABAMA,Marshall County,10000600876,Claysville Jr High Sch,36,20,19,21,17,20,NaN,NaN,NaN,NaN


#### 2. Create class variables in school directory dataset

If a school offers grade 12 and not grade 1 it is considered a High School. If a school offers grade 1 but not grade 8 it is considered elementary. If a school offers grade 8 but not 12 or 1, it is a middle school

In [22]:
#G12OFFERED = YES then this is a high school
#G1OFFERED = YES and G12OFFERED=NO then this is an elementary school
#first replace yes and no with 1 and 0, respectively

school_dir=school_dir.replace(['yes', 'Yes'], 1, regex=True)
#school_dir1=school_dir.replace('Yes', 1)
school_dir=school_dir.replace('No', 0, regex=True)

In [23]:
school_dir['type_school']=np.nan

#df['elderly'] = np.where(df['age']>=50, 'yes', 'no')
      
school_dir['type_school'] = np.where((school_dir['G1OFFERED']==1) & (school_dir['G12OFFERED']==0) , 'elementary',
                np.where((school_dir['G1OFFERED']==0) & (school_dir['G12OFFERED']==1) , 'high_school', 
                    np.where((school_dir['G1OFFERED']==0) & (school_dir['G12OFFERED']==0)& (school_dir['G8OFFERED']==1) , 'middle',  'unknown')))


In [24]:
school_dir[['G1OFFERED', 'G8OFFERED', 'G12OFFERED',  'type_school']].head(10)

,G1OFFERED,G8OFFERED,G12OFFERED,type_school
0,0,1,1,high_school
1,0,0,0,unknown
2,0,1,1,high_school
3,0,1,1,high_school
4,0,1,1,high_school
5,0,1,1,high_school
6,0,0,0,unknown
7,0,0,0,unknown
8,0,1,0,middle
9,0,0,0,unknown


In [25]:
#Change all the Cohort variables (those with specific numbers of students to float instead of object)
r=re.compile(".*COHORT")
cohort_variables = list(filter(r.match, grad_variables))

for variable in rate_variables:
    grad_2015[variable]=grad_2015[variable].astype('float')
    
for variable in cohort_variables:
    grad_2015[variable]=grad_2015[variable].astype('float')

In [26]:
grad_2015.describe() #everything we would expect is numeric and can be summarized. These columns can also be used to creat a distance matrix between the different schools.

,NCESSCH,ALL_COHORT_1516,ALL_RATE_1516,MAM_RATE_1516,MAS_RATE_1516,MBL_RATE_1516,MHI_RATE_1516,MTR_RATE_1516,MWH_RATE_1516,CWD_RATE_1516,ECD_RATE_1516,LEP_RATE_1516
count,2.309000e+04,23090.00000,21537.000000,1400.000000,5250.000000,9858.000000,11900.000000,4418.000000,18344.000000,15049.000000,20125.000000,6279.000000
mean,2.779916e+11,158.25929,76.398106,55.506429,67.515429,66.321668,65.982353,58.618379,76.838203,57.239883,69.643776,56.932155
std,1.595166e+11,179.91495,21.973247,14.389704,18.621002,20.036679,20.865298,14.619950,19.856361,16.321042,20.553885,18.175620
min,1.000050e+10,1.00000,1.000000,6.000000,10.000000,1.000000,2.000000,5.000000,5.000000,5.000000,2.000000,2.000000
25%,1.201980e+11,31.00000,70.000000,50.000000,50.000000,50.000000,50.000000,50.000000,60.000000,50.000000,50.000000,50.000000
50%,2.723535e+11,87.00000,85.000000,50.000000,60.000000,70.000000,70.000000,50.000000,85.000000,50.000000,80.000000,50.000000
75%,4.025388e+11,235.00000,90.000000,60.000000,80.000000,80.000000,80.000000,80.000000,90.000000,70.000000,85.000000,70.000000
max,5.900197e+11,4713.00000,99.000000,95.000000,99.000000,98.000000,99.000000,95.000000,99.000000,95.000000,99.000000,95.000000


Clustering algorithims cannot handle missing data, what data is missing?

In [27]:
na_list=grad_2015.isna().sum()
na_list/len(grad_2015) # percentage
#Because many of these race groups are so small the PS identifier was used, and then converted to NA.

NCESSCH            0.000000
ALL_COHORT_1516    0.000000
ALL_RATE_1516      0.067259
MAM_RATE_1516      0.939368
MAS_RATE_1516      0.772629
MBL_RATE_1516      0.573062
MHI_RATE_1516      0.484625
MTR_RATE_1516      0.808662
MWH_RATE_1516      0.205544
CWD_RATE_1516      0.348246
ECD_RATE_1516      0.128411
LEP_RATE_1516      0.728064
dtype: float64

In [28]:
# Fill missing values with mean column values in those columns
grad_2015.fillna(grad_2015.mean(), inplace=True)

In [29]:
suspensions.fillna(suspensions.mean(), inplace=True)

#### All NAs are removed from the grad_2015 dataset:

In [30]:
grad_2015.isna().sum()/len(grad_2015) #all NAS removed

NCESSCH            0.0
ALL_COHORT_1516    0.0
ALL_RATE_1516      0.0
MAM_RATE_1516      0.0
MAS_RATE_1516      0.0
MBL_RATE_1516      0.0
MHI_RATE_1516      0.0
MTR_RATE_1516      0.0
MWH_RATE_1516      0.0
CWD_RATE_1516      0.0
ECD_RATE_1516      0.0
LEP_RATE_1516      0.0
dtype: float64

#### ALL NA's removed from reading and math assessment data:

In [31]:
math.fillna(0, inplace=True) #replace those with no data with 0
reading.fillna(0, inplace=True)

In [32]:
math.iloc[:,4:]=math.iloc[:,4:].astype(str).astype(int)

In [33]:
math.isna().sum()

STNAM                     0
LEANM                     0
NCESSCH                   0
SCHNAM                    0
ALL_MTH00NUMVALID_1516    0
ALL_MTH00PCTPROF_1516     0
MWH_MTH00PCTPROF_1516     0
MBL_MTH00PCTPROF_1516     0
F_MTH00NUMVALID_1516      0
F_MTH00PCTPROF_1516       0
M_MTH00NUMVALID_1516      0
M_MTH00PCTPROF_1516       0
MBL_MTHHSNUMVALID_1516    0
MBL_MTHHSPCTPROF_1516     0
MWH_MTHHSNUMVALID_1516    0
MWH_MTHHSPCTPROF_1516     0
dtype: int64

#### All NAs are removed from the school_member dataset:

In [34]:
school_member=school_member.replace(-1,0)
school_member.head(8)

,NCESSCH,SCH_NAME,G01,G08,G12,TOTAL,AM,AMALM,AMALF,AS,...,BLALF,WH,WHALM,WHALF,HP,HPALM,HPALF,TR,TRALM,TRALF
0,10000200277,Sequoyah Sch - Chalkville Campus,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000201667,Camps,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10000201670,Det Ctr,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000201705,Wallace Sch - Mt Meigs Campus,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000201706,McNeel Sch - Vacca Campus,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,10000500870,Ala Avenue Middle Sch,0,333,0,704,3,3,0,1,...,6,385,196,189,0,0,0,23,11,12
6,10000500871,Albertville High Sch,0,0,282,1290,2,0,2,6,...,18,762,360,402,2,0,2,12,6,6
7,10000500879,Evans Elem Sch,0,0,0,766,1,1,0,4,...,13,399,200,199,1,0,1,13,8,5


#### Suspension NAs are removed

In [35]:
suspensions.isna().sum()

School_ID                 0
TOT_n_susp                0
TOT_OSS_rate              0
BLA_OSS_rate              0
WHI_OSS_rate              0
AME_OSS_times_white       0
ASI_OSS_times_white       0
HI_PAC_OSS_times_white    0
BLA_OSS_times_white       0
HIS_OSS_times_white       0
TWO_OSS_times_white       0
TOT_n_enrollment          0
dtype: int64

## Merge all the data for individual schools together, using 'NCESSCH' as the key id variable.

In [36]:
#before we merge data, lets rename some of the variables so they do not get confused

school_member= school_member.add_suffix('_n_students')
school_member.rename(columns={'NCESSCH_n_students': 'NCESSCH'}, inplace=True)
school_member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99264 entries, 0 to 99263
Data columns (total 27 columns):
NCESSCH                99264 non-null int64
SCH_NAME_n_students    99264 non-null object
G01_n_students         99264 non-null int64
G08_n_students         99264 non-null int64
G12_n_students         99264 non-null int64
TOTAL_n_students       99264 non-null int64
AM_n_students          99264 non-null int64
AMALM_n_students       99264 non-null int64
AMALF_n_students       99264 non-null int64
AS_n_students          99264 non-null int64
ASALM_n_students       99264 non-null int64
ASALF_n_students       99264 non-null int64
HI_n_students          99264 non-null int64
HIALM_n_students       99264 non-null int64
HIALF_n_students       99264 non-null int64
BL_n_students          99264 non-null int64
BLALM_n_students       99264 non-null int64
BLALF_n_students       99264 non-null int64
WH_n_students          99264 non-null int64
WHALM_n_students       99264 non-null int64
WHALF_n_st

In [84]:
#grad_2015 = grad_2015.rename(columns={col: col.split('_')[1] for col in grad_2015.columns})

grad_2015=grad_2015.rename(columns = lambda x : str(x)[:-5])
grad_2015.rename(columns={'NC': 'NCESSCH'}, inplace=True)
grad_2015.head()

,NCESSCH,ALL_COHORT,ALL_RATE,MAM_RATE,MAS_RATE,MBL_RATE,MHI_RATE,MTR_RATE,MWH_RATE,CWD_RATE,ECD_RATE,LEP_RATE
0,10000500871,296.0,92.0,55.506429,67.515429,50.000000,95.000000,58.618379,90.0,60.0,80.0,50.000000
1,10000600872,67.0,95.0,55.506429,67.515429,66.321668,80.000000,58.618379,90.0,50.0,90.0,56.932155
2,10000600878,153.0,85.0,55.506429,67.515429,66.321668,90.000000,58.618379,85.0,50.0,85.0,56.932155
3,10000600883,120.0,80.0,55.506429,67.515429,66.321668,65.982353,58.618379,80.0,21.0,60.0,56.932155
4,10000601585,94.0,85.0,55.506429,67.515429,66.321668,65.982353,58.618379,85.0,50.0,80.0,56.932155


In [85]:
math2=math
math2=math2.rename(columns=lambda x: str(x) [:-5])

objs=[math.iloc[:,0:3], math2.iloc[:,4:]]

math=pd.concat(objs, axis=1)


In [86]:
reading2=reading
reading2=reading2.rename(columns=lambda x: str(x) [:-5])

objs=[reading.iloc[:,0:3], reading2.iloc[:,4:]]
reading=pd.concat(objs, axis=1)
reading.head()

,STNAM,LEANM,NCESSCH,ALL_RLA00NUMVALID,ALL_RLA00PCTPROF,F_RLA00NUMVALID,F_RLA00PCTPROF,M_RLA00NUMVALID,M_RLA00PCTPROF,MBL_RLAHSNUMVALID,MBL_RLAHSPCTPROF,MWH_RLAHSNUMVALID,MWH_RLAHSPCTPROF
0,ALABAMA,Albertville City,10000500870,687,33,339,39,348,28,0,0,0,0
1,ALABAMA,Albertville City,10000500871,307,31,162,30,145,25,9,50,178,35
2,ALABAMA,Albertville City,10000500879,759,30,381,36,378,23,0,0,0,0
3,ALABAMA,Albertville City,10000500889,839,36,414,37,425,34,0,0,0,0
4,ALABAMA,Marshall County,10000600193,442,42,205,45,237,35,0,0,0,0


In [87]:
#suspensions.columns[0]
suspensions.rename(columns={'School_ID': 'NCESSCH'}, inplace=True)

In [88]:
all_data=pd.merge(school_dir, school_member, on='NCESSCH') #this is an inner join, and any schools without info in either dataset are removed

In [89]:
all_data=pd.merge(all_data, school_staff, on='NCESSCH')
all_data=pd.merge(all_data, reading, on='NCESSCH')
all_data=pd.merge(all_data, math, on='NCESSCH')

grad_all_data=pd.merge(all_data, grad_2015, on='NCESSCH') # grad_2015 only includes highschools
suspensions_all_data=pd.merge(all_data, suspensions, on='NCESSCH') #suspensions only include greenville data
suspensions_grad_greenville=pd.merge(grad_all_data, suspensions, on='NCESSCH')#this is graduation rates in high schools with suspensions

In [90]:
all_data.head()

,FIPST,NCESSCH,LCITY,LZIP,G1OFFERED,G8OFFERED,G12OFFERED,NOGRADES,type_school,SCH_NAME_n_students,...,MWH_MTH00PCTPROF,MBL_MTH00PCTPROF,F_MTH00NUMVALID,F_MTH00PCTPROF,M_MTH00NUMVALID,M_MTH00PCTPROF,MBL_MTHHSNUMVALID,MBL_MTHHSPCTPROF,MWH_MTHHSNUMVALID,MWH_MTHHSPCTPROF
0,1,10000500889,Albertville,35950,0,0,0,0,unknown,Albertville Elem Sch,...,67,20,421,54,431,57,0,0,0,0
1,1,10000500870,Albertville,35950,0,1,0,0,middle,Ala Avenue Middle Sch,...,45,20,347,39,360,34,0,0,0,0
2,1,10000500879,Albertville,35950,0,0,0,0,unknown,Evans Elem Sch,...,52,21,389,46,383,40,0,0,0,0
3,1,10000500871,Albertville,35950,0,0,1,0,high_school,Albertville High Sch,...,20,50,161,15,146,10,9,50,177,20
4,1,10000600876,Guntersville,35976,1,0,0,0,elementary,Claysville Jr High Sch,...,40,0,20,40,17,21,0,0,0,0


In [91]:
grad_all_data.head()

,FIPST,NCESSCH,LCITY,LZIP,G1OFFERED,G8OFFERED,G12OFFERED,NOGRADES,type_school,SCH_NAME_n_students,...,ALL_RATE,MAM_RATE,MAS_RATE,MBL_RATE,MHI_RATE,MTR_RATE,MWH_RATE,CWD_RATE,ECD_RATE,LEP_RATE
0,1,10000500871,Albertville,35950,0,0,1,0,high_school,Albertville High Sch,...,92.0,55.506429,67.515429,50.000000,95.000000,58.618379,90.0,60.0,80.0,50.000000
1,1,10000600872,Albertville,35951,0,1,1,0,high_school,Asbury Sch,...,95.0,55.506429,67.515429,66.321668,80.000000,58.618379,90.0,50.0,90.0,56.932155
2,1,10000601585,Guntersville,35976,0,0,1,0,high_school,Brindlee Mt High Sch,...,85.0,55.506429,67.515429,66.321668,65.982353,58.618379,85.0,50.0,80.0,56.932155
3,1,10000600883,Grant,35747,0,0,1,0,high_school,Kate D Smith DAR High Sch,...,80.0,55.506429,67.515429,66.321668,65.982353,58.618379,80.0,21.0,60.0,56.932155
4,1,10000600878,Douglas,35964,0,0,1,0,high_school,Douglas High Sch,...,85.0,55.506429,67.515429,66.321668,90.000000,58.618379,85.0,50.0,85.0,56.932155


In [92]:
suspensions_all_data.head()

,FIPST,NCESSCH,LCITY,LZIP,G1OFFERED,G8OFFERED,G12OFFERED,NOGRADES,type_school,SCH_NAME_n_students,...,TOT_OSS_rate,BLA_OSS_rate,WHI_OSS_rate,AME_OSS_times_white,ASI_OSS_times_white,HI_PAC_OSS_times_white,BLA_OSS_times_white,HIS_OSS_times_white,TWO_OSS_times_white,TOT_n_enrollment
0,45,450231001633,Greenville,29607,0,0,0,0,unknown,Dr. Phinnize J. Fisher Middle,...,13.48,24.88,4.76,0.00,1.5,0.0,5.23,4.80,8.85,663
1,45,450231001593,Simpsonville,29681,1,0,0,0,elementary,Monarch Elementary,...,1.15,0.00,0.88,0.00,0.0,0.0,0.00,10.82,0.00,895
2,45,450231001524,Simpsonville,29680,0,1,0,0,middle,Ralph Chandler Middle,...,3.31,6.06,2.70,0.00,0.0,0.0,2.24,1.69,2.31,695
3,45,450231001521,Taylors,29687,0,0,1,0,high_school,Greer Middle College Charter,...,1.88,0.00,2.15,0.00,0.0,0.0,0.00,0.00,0.00,440
4,45,450231001517,Simpsonville,29681,1,0,0,0,elementary,Rudolph Gordon Elementary,...,3.45,11.76,1.82,27.47,0.0,0.0,6.46,4.48,3.92,797


## Export the cleaned data for further analysis

In [93]:
all_data.to_csv('CleanedData/all_data.csv')
grad_all_data.to_csv('CleanedData/grad_all_data.csv')
suspensions_all_data.to_csv('CleanedData/suspensions_all_data.csv')
suspensions_grad_greenville.to_csv('CleanedData/suspensions_grad_greenville.csv')